<a href="https://colab.research.google.com/github/tlenzmeier58/big_data_challenge/blob/main/working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 import os
 import pandas as pd
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

#sys.path.append("/opt/spark-3.2.1-bin-hadoop3.2/python")
#sys.path.append("/opt/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.4-src.zip")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:1

In [2]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-29 15:10:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-11-29 15:10:34 (10.6 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import modules
from pyspark import SparkFiles

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [4]:
from posixpath import sep
#Load data
base_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/"
file = "amazon_reviews_us_Watches_v1_00.tsv.gz"
url = base_url + file
spark.sparkContext.addFile(url)

# Initial DF
df = spark.read.option('header', 'true').csv(SparkFiles.get(file),
                                             inferSchema=True,
                                             sep='\t',
                                             timestampFormat = "yyyy-mm-dd")
df.show(truncate=True)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-01-31 00:08:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [5]:
# Cleaning time . . .
df = df.dropna()
df = df.dropDuplicates()

In [6]:
# Row count
df.count()

960679

In [7]:
#Schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
 # IMPORTANT: Replace each of these parameters with your own values for your AWS RDS instance
my_aws_endpoint = 'big-data-challenge.ch9chhy09aqn.us-west-2.rds.amazonaws.com' # This is my value; please replace with your own
my_aws_port_number = '5432' # Your value is likely the same, but please double check
my_aws_database_name = 'big_data_etl' # This is my value; please replace with your own
my_aws_username = 'postgres' # Your value is likely the same, but please double check
my_aws_password = 'XXXXXXXX' # This is my value; please replace with your own

 # Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

In [ ]:
 # Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}


mode = 'overwrite' 

In [8]:
# Watches df
#watches = df[['product_id', 'product_parent', 'product_title']]
watches = df.select(['product_id', 'product_parent', 'product_title'])
watches.show(truncate=True)



# Push to PostgreSQL

watches.write.jdbc(url = jdbc_url, table = 'watches', mode = mode, properties = config)


+----------+--------------+--------------------+
|product_id|product_parent|       product_title|
+----------+--------------+--------------------+
|B002XUL6YY|     852892099|Rip Curl Men's A1...|
|B005JVP0LE|     559701518|Casio Men's Sport...|
|B0009P66YK|       6220965|Invicta Men's 901...|
|B008MVVA2Q|     849986272|U.S. Polo Assn. C...|
|B001D9FGAG|     940154187|Invicta Men's 524...|
|B007I1PLUI|     906701484|Rudiger Men's R20...|
|B0041IEATO|     950717807|Invicta Women's 0...|
|B000JQSMQO|     963972840|Fossil Men's Blue...|
|B001CTI5SM|     243892088|Invicta Men's 505...|
|B00JMPN3RQ|     849006421|Swiss Legend Men'...|
|B000K6HILI|      13227568|Invicta Women's 3...|
|B0051UMEHG|     732465657|Hadley Roma MS834...|
|B004IBT2D8|     946540922|Geneva Platinum W...|
|B00EAU0IDA|      60629838|Compact Watch Bra...|
|B000SSPML6|     722719968|Seiko Men's SGG71...|
|B0046CIYAG|     420607996|Roberto Bianci Me...|
|B004RQ5A4O|     835018862|Watch Band Rubber...|
|B006KY5TLK|     369

In [9]:
watches.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)



In [10]:
# Reviews df
watch_reviews = df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
watch_reviews.show(truncate=True)

watch_reviews.write.jdbc(url=jdbc_url,
                   table = 'watch_reviews',
                   mode = mode,
                   properties = config)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1004K426LTDKN|   15688375|B002XUL6YY|     852892099|2014-01-22 00:05:00|
|R1006QGKXQ3Q96|   21739087|B005JVP0LE|     559701518|2014-01-16 00:06:00|
|R1017P0ZGRXSR0|    2501262|B0009P66YK|       6220965|2015-01-26 00:02:00|
|R103HXXDQD9W4Y|   16842563|B008MVVA2Q|     849986272|2014-01-30 00:12:00|
|R10472PWW2S2AW|   52130705|B001D9FGAG|     940154187|2009-01-27 00:01:00|
|R104EYLG2ZXXJ1|   41518340|B007I1PLUI|     906701484|2014-01-27 00:11:00|
|R105KF16VGOILR|   48845798|B0041IEATO|     950717807|2011-01-02 00:09:00|
|R106A2ETV5NZNE|   50381801|B000JQSMQO|     963972840|2012-01-16 00:12:00|
|R106UPPPFIIX3P|   42038724|B001CTI5SM|     243892088|2013-01-17 00:08:00|
|R107IKXXNUAZZC|    1594613|B00JMPN3RQ|     849006421|2015-01-11 00:08:00|
|R1081CHIZX3PZW|   459864

In [11]:
watch_reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [12]:
# Customers
watch_customers = df.select(['customer_id'])
watch_customers = watch_customers.groupBy('customer_id').count()
watch_customers = watch_customers.withColumnRenamed('count','customer_count')
watch_customers.show(truncate=True)

# Push to PostgreSQL
watch_customers.write.jdbc(url=jdbc_url,
                     table = 'watch_customers',
                     mode=mode,
                     properties = config)


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45353385|             1|
|    7166154|             1|
|   27053878|             1|
|   33656295|             1|
|   34320055|             1|
|   12611685|             1|
|   26417168|             5|
|   16885399|             3|
|   49090853|             1|
|   48080283|             1|
|   12937506|             5|
|   42209526|             1|
|   36125184|             1|
|   50308356|             1|
|   13273143|             1|
|   30675744|             1|
|   10196801|             3|
|   10748967|             1|
|   44443429|             1|
|   15687327|             2|
+-----------+--------------+
only showing top 20 rows



In [14]:
watch_customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)

